In [1]:
import cv2 as cv
import numpy as np
import imutils
import time
import random
from shapely.geometry import Polygon, Point
from matplotlib.path import Path
import os

def resize_image(image, scale_percent):
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
  
    # resize image
    resized = cv.resize(image, dim, cv.INTER_AREA)
    return resized

def get_contours(image):
    contours, hierarchy = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    big_contours = [cnt for cnt in contours if cv.contourArea(cnt) > 1000]
    return big_contours


def rotate_image(image, degrees):
    rotated = imutils.rotate_bound(image, degrees)
    return rotated


def grabCut_image(image, mask):
    start = time.time()
    fgModel = np.zeros((1, 65), dtype="float")
    bgModel = np.zeros((1, 65), dtype="float")
    mask[mask > 0] = cv.GC_PR_FGD
    mask[mask == 0] = cv.GC_BGD
    (mask, bgModel, fgModel) = cv.grabCut(src, mask, None, bgModel,fgModel, iterCount=5, mode=cv.GC_INIT_WITH_MASK)
    
    values = (
        ("Definite Background", cv.GC_BGD),
        ("Probable Background", cv.GC_PR_BGD),
        ("Definite Foreground", cv.GC_FGD),
        ("Probable Foreground", cv.GC_PR_FGD),
    )
    valueMask = (mask == values[3][1]).astype("uint8") * 255
    end = time.time()
    
    print(f"took: {end-start}")
    
    return valueMask

def get_pill_sides(image, mask):
    # get a refined contour
    contours = get_contours(mask)
    
    # array to hold each side masks
    side_masks = [mask.copy(), mask.copy()]
    sides = []
    for side_mask, cnt in zip(side_masks, contours):
        
        cv.fillPoly(side_mask, pts=[cnt], color=(0,0,0))
        
        side_masked_img = cv.bitwise_and(image, image, mask=side_mask)
        
        sides.insert(0, side_masked_img)
        
    x,y,w,h = cv.boundingRect(contours[0])
    sides[0] = crop_bounding_box(sides[0], x,y,w,h)
    
    x,y,w,h = cv.boundingRect(contours[1])
    sides[1] = crop_bounding_box(sides[1], x,y,w,h)
    
    return sides, contours
    
    
def make_black_transparent(image):
    image_BGRA = cv.cvtColor(image, cv.COLOR_BGR2BGRA)
    black_mask = np.all(image == [0, 0, 0], axis=-1)
    image_BGRA[black_mask, -1] = 0
    
    return image_BGRA
    

def paste_over(image, background, x=0, y=0):
    
    # Get both dimensions
    img_h, img_w, _ = image.shape
    
    max_h, max_w, _ = background.shape
    
    # Checks if the image will be out of bounds and clips it
    if x + img_w > max_w:
        img_w = max_w - x -1
    if y + img_h > max_h:
        img_h = max_h - y -1
    
    if img_w == 0 or img_h == 0:
        raise ValueError("Either w is 0 or h is 0")
    
    for i in range(img_h):
        for j in range(img_w):
            if image[i][j].all(0):
                background[i+y][j+x] = image[i][j]
             
    return background, img_h, img_w



def create_annotations(bounding_box_list, max_w, max_h):
    print_buffer = []
    for b in bounding_box_list:
        # clip the max value so its <= the background
        b_max_x = (b[0]+b[2]) if (b[0]+b[2]) < max_w else max_w
        b_max_y = (b[1]+b[3]) if (b[1]+b[3]) < max_h else max_h
        
        # get the center of each bound box 
        b_center_x = (b[0] + b_max_x) / 2 / max_w
        b_center_y = (b[1] + b_max_y) / 2 / max_h
        
        # convert width and height of BB to %
        b_width = b[2] / max_w
        b_height = b[3] / max_h
        
        # String to store file
        f_name = "{} {:.3f} {:.3f} {:.3f} {:.3f}".format(0, b_center_x, b_center_y, b_width, b_height)
        
        print_buffer.append(f_name)
    return print_buffer

def safe_annotations(name, print_buffer):
    with open("{}.txt".format(name) , "w") as _file:
        for line in print_buffer:
            _file.write(line+"\n")

        _file.close()

In [2]:
class Background:  
    def __init__(self, path):
        self.file_list = []
        for root, dirs, files in os.walk(path):
            for file in files:
                if file.endswith(".ppm"):
                     self.file_list.append(os.path.join(root, file))
    
    def get_random_image(self):
        r = random.randint(0, len(self.file_list)-1)
        return self.file_list[r]

In [3]:
class Pill_Generator:
    def __init__(self, path):
        self.__file_list = []
        for root, dirs, files in os.walk(path):
            for file in files:
                if file.endswith(".jpg"):
                     self.__file_list.append(os.path.join(root, file))
                        
    def get_random_pill(self):
        while True:
            try:
                r = random.randint(0, (len(self.__file_list)-1))
                pill = Pill(self.__file_list[r])
                break
            except:
                print("retrying pill contours")
        return pill

In [4]:
class Pill:
    
    def __init__(self, path):
        self.image = cv.imread(path)
        self.mask = self.preprocess()
        self.cnt = self.get_contours()
        cv.fillPoly(self.mask, pts=self.cnt, color=(255,255,255))
        self.mask = self.grab_cut();
        self.sides = self.get_sides_of_pills()
        
    def preprocess(self):
        image =  cv.fastNlMeansDenoisingColored(self.image.copy(),None,10,10,3,3)
        hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
        mask = cv.inRange(hsv, (0,4, 0), (179, 255, 255))
        kernel = cv.getStructuringElement(cv.MORPH_RECT, (3,3))

        dilated = cv.dilate(mask, None, iterations=5)
        eroded = cv.erode(dilated, None, iterations=8)

        blurred = cv.GaussianBlur(eroded,(9,9),11)
        return blurred
        
    def get_contours(self, limit=1000):
        _, cnt_mask = cv.threshold(self.mask,0,255,cv.THRESH_BINARY)
        #cnt_mask = cv.cvtColor(cnt_mask, cv.COLOR_RGB2GRAY)
        cnt, _ = cv.findContours(cnt_mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

        if (limit!= 1000):
            cnt = [cnt for cnt in contours if cv.contourArea(cnt) > limit]
        
        return cnt
    
    def grab_cut(self):
        fgModel = np.zeros((1, 65), dtype="float")
        bgModel = np.zeros((1, 65), dtype="float")
        self.mask[self.mask > 0] = cv.GC_PR_FGD
        self.mask[self.mask == 0] = cv.GC_BGD
        (mask, bgModel, fgModel) = cv.grabCut(self.image, self.mask, None, bgModel,fgModel, iterCount=5, mode=cv.GC_INIT_WITH_MASK)

        values = (
            ("Definite Background", cv.GC_BGD),
            ("Probable Background", cv.GC_PR_BGD),
            ("Definite Foreground", cv.GC_FGD),
            ("Probable Foreground", cv.GC_PR_FGD),
        )
        valueMask = (mask == values[3][1]).astype("uint8") * 255
    
        return valueMask
    
    def get_sides_of_pills(self):
        def crop_bounding_box(image, x, y, w, h):
            # crop the image so its just the pills
            image = image[y:y+h, x:x+w]

            return image
        
        # get a refined contour
        contours = get_contours(self.mask)

        # array to hold each side masks
        side_masks = [self.mask.copy(), self.mask.copy()]
        sides = []
        for side_mask, cnt in zip(side_masks, contours):

            cv.fillPoly(side_mask, pts=[cnt], color=(0,0,0))

            side_masked_img = cv.bitwise_and(self.image, self.image, mask=side_mask)

            sides.insert(0, side_masked_img)

        x,y,w,h = cv.boundingRect(contours[0])
        sides[0] = crop_bounding_box(sides[0], x,y,w,h)

        x,y,w,h = cv.boundingRect(contours[1])
        sides[1] = crop_bounding_box(sides[1], x,y,w,h)

        return sides

In [5]:
class Image_Generator:
    def __init__(self, b, pill):
        self.b = b
        self.pill = pill
        
    def __get_cunts(self, img):
        _, mask = cv.threshold(img,0,255,cv.THRESH_BINARY)
        mask = cv.cvtColor(mask, cv.COLOR_RGB2GRAY)
        cnt, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        return cnt
    
    def get_image(self):
        while True:
            try:
                pill = pg.get_random_pill()
                break
            except:
                print("retrying pill contours")

        background = cv.imread(self.b.get_random_image(), cv.IMREAD_UNCHANGED)

        background = cv.resize(background, (512,512), cv.INTER_AREA)

        size = random.randint(10, 20)

        resize = []
        resize.append(resize_image(self.pill.sides[0], size))
        resize.append(resize_image(self.pill.sides[1], size))

        max_h, max_w, _ = background.shape

        error_count = 0
        self.num_pills = 0
        contour_list = []
        self.bounding_box_list = []
        poly_list = []
        pill_limit = 100

        for i in range(pill_limit):
            if(error_count > 5):
                break

            internal_error = 0

            while True:

                if internal_error > pill_limit:
                    print("Too many errors; giving up")
                    break

                x = random.randint(0, max_w-25)
                y = random.randint(0, max_h-25)
                degree = random.randint(0, 359)
                rand_side = random.randint(0, 1)

                img = rotate_image(resize[rand_side], degree)
                cnt = self.__get_cunts(img)[0] + [x, y]    
                cnt_poly = Polygon(np.squeeze(cnt))

                if not any(cnt_poly.overlaps(p) for p in poly_list):
                    break

                internal_error += 1

            if internal_error > pill_limit:
                 break

            try:
                background, img_h, img_w = paste_over(img, background, x, y)
                contour_list.append(cnt)
                poly_list.append(cnt_poly)
                # add (x,y) transltion but also clip the ends if they are out of range
                self.bounding_box_list.append(cv.boundingRect(cnt))

                self.num_pills += 1
            except ValueError:
                error_count += 1
        
        return background

In [ ]:
"""# read the image
src = cv.imread('test1.jpg')

# Get the contour masks
mask = preprocess_image(src)
contours = get_contours(mask)

# fill the contour completley white
cv.fillPoly(mask, pts=contours, color=(255,255,255))

# get a more refined mask from grabCut
mask = grabCut_image(src, mask)


sides, contours = get_pill_sides(src, mask)"""

In [26]:
import multiprocess as mp
pool = mp.Pool(mp.cpu_count())

b = Background(r".\src_img\VisionTexture")
pg = Pill_Generator(r".\src_img\pills")


dg = Data_Generator(r"./data")
 
    
for i in range(1000):
    try:
        pill = pg.get_random_pill()
        ig = Image_Generator(b, pill) 
        image = ig.get_image()
        dg.save_data(image, ig.bounding_box_list, "IM"+str(i))
        print(f"done: {i}")
    except:
        i -= 1
        print(f"Failed {i} retrying")

Too many errors; giving up
done: 0
Too many errors; giving up
done: 1
retrying pill contours
done: 2
retrying pill contours
Too many errors; giving up
done: 3
done: 4
Too many errors; giving up
done: 5
Too many errors; giving up
done: 6
Too many errors; giving up
done: 7
done: 8
Too many errors; giving up
done: 9
retrying pill contours
Failed 9 retrying
done: 11
retrying pill contours
retrying pill contours
Too many errors; giving up
done: 12
Too many errors; giving up
done: 13
retrying pill contours
Too many errors; giving up
done: 14
Too many errors; giving up
done: 15
Too many errors; giving up
done: 16
Too many errors; giving up
done: 17
retrying pill contours
Too many errors; giving up
done: 18
Too many errors; giving up
done: 19
Failed 19 retrying
retrying pill contours
Too many errors; giving up
done: 21
Too many errors; giving up
done: 22
retrying pill contours
retrying pill contours
Too many errors; giving up
done: 23
Too many errors; giving up
done: 24
Too many errors; giving

TopologyException: side location conflict at 135 80


Failed 110 retrying
Too many errors; giving up
done: 112
Too many errors; giving up
done: 113
retrying pill contours
done: 114
done: 115
Too many errors; giving up
done: 116
Too many errors; giving up
done: 117
Too many errors; giving up
done: 118
retrying pill contours
Too many errors; giving up
done: 119
done: 120
done: 121
Too many errors; giving up
done: 122
Too many errors; giving up
done: 123
Too many errors; giving up
done: 124
retrying pill contours
Too many errors; giving up
done: 125
Too many errors; giving up
done: 126
retrying pill contours
Too many errors; giving up
done: 127
retrying pill contours
done: 128
done: 129
Too many errors; giving up
done: 130
done: 131
retrying pill contours
done: 132
retrying pill contours
Too many errors; giving up
done: 133
done: 134
Too many errors; giving up
done: 135
done: 136
Too many errors; giving up
done: 137
retrying pill contours
Too many errors; giving up
done: 138
done: 139
Too many errors; giving up
done: 140
Too many errors; giv

TopologyException: side location conflict at 124 75


Failed 143 retrying
done: 145
Too many errors; giving up
done: 146
Too many errors; giving up
done: 147
Too many errors; giving up
done: 148
Too many errors; giving up
done: 149
done: 150
retrying pill contours
done: 151
done: 152
retrying pill contours
retrying pill contours
Too many errors; giving up
done: 153
done: 154
retrying pill contours
done: 155


...

Failed 172 retrying
Too many errors; giving up
done: 174
retrying pill contours
Too many errors; giving up
done: 175
done: 176
Too many errors; giving up
done: 177
retrying pill contours
Too many errors; giving up
done: 178
Too many errors; giving up
done: 179
Too many errors; giving up
done: 180
retrying pill contours
Too many errors; giving up
done: 181
done: 182
Too many errors; giving up
done: 183
Too many errors; giving up
done: 184
Too many errors; giving up
done: 185
done: 186
done: 187
Too many errors; giving up
done: 188
Too many errors; giving up
done: 189
Too many errors; giving up
done: 190
done: 191
retrying pill contours
done: 192
Too many errors; giving up
done: 193
Too many errors; giving up
done: 194
Too many errors; giving up
done: 195
Too many errors; giving up
done: 196
done: 197
Too many errors; giving up
done: 198
Too many errors; giving up
done: 199
Too many errors; giving up
done: 200
Too many errors; giving up
done: 201
Too many errors; giving up
done: 202
done

TopologyException: side location conflict at 159 379


Failed 328 retrying
Too many errors; giving up
done: 330
Too many errors; giving up
done: 331
done: 332
done: 333
Too many errors; giving up
done: 334
Too many errors; giving up
done: 335
done: 336
done: 337
retrying pill contours
done: 338
Too many errors; giving up
done: 339
retrying pill contours
Too many errors; giving up
done: 340
done: 341
retrying pill contours
done: 342
Too many errors; giving up
done: 343
Too many errors; giving up
done: 344
Too many errors; giving up
done: 345
retrying pill contours
retrying pill contours
retrying pill contours
Too many errors; giving up
done: 346
Too many errors; giving up
done: 347
retrying pill contours
done: 348
Too many errors; giving up
done: 349
done: 350
done: 351
Too many errors; giving up
done: 352
Too many errors; giving up
done: 353
Too many errors; giving up
done: 354
Too many errors; giving up
done: 355
Too many errors; giving up
done: 356
done: 357
retrying pill contours
done: 358
done: 359
retrying pill contours
retrying pill 

TopologyException: side location conflict at 118 89


Failed 688 retrying
done: 690
retrying pill contours
done: 691
Too many errors; giving up
done: 692
Too many errors; giving up
done: 693
Too many errors; giving up
done: 694
Too many errors; giving up
done: 695
Failed 695 retrying
Too many errors; giving up
done: 697
Too many errors; giving up
done: 698
Too many errors; giving up
done: 699
Too many errors; giving up
done: 700
Too many errors; giving up
done: 701
Too many errors; giving up
done: 702
Too many errors; giving up
done: 703
Too many errors; giving up
done: 704
Too many errors; giving up
done: 705
Too many errors; giving up
done: 706
done: 707
retrying pill contours
Too many errors; giving up
done: 708
Too many errors; giving up
done: 709
Too many errors; giving up
done: 710
done: 711
Too many errors; giving up
done: 712
Failed 712 retrying
Too many errors; giving up
done: 714
Too many errors; giving up
done: 715
Too many errors; giving up
done: 716
retrying pill contours
retrying pill contours
Too many errors; giving up
done

In [ ]:
print(ig.num_pills)
cv.imshow("Rotated Without Cropping", image)
cv.waitKey(0)

In [6]:
class Data_Generator:
    
    def __init__(self, path):
        self.path = path
    
    def __check_path_exists(self, path):
        if not os.path.exists(path):
            os.makedirs(path)
            
            
    def create_annotations(self, bounding_box_list, max_w=512, max_h=512):
        print_buffer = []
        for b in bounding_box_list:
            # clip the max value so its <= the background
            b_max_x = (b[0]+b[2]) if (b[0]+b[2]) < max_w else max_w
            b_max_y = (b[1]+b[3]) if (b[1]+b[3]) < max_h else max_h

            # get the center of each bound box 
            b_center_x = (b[0] + b_max_x) / 2 / max_w
            b_center_y = (b[1] + b_max_y) / 2 / max_h

            # convert width and height of BB to %
            b_width = b[2] / max_w
            b_height = b[3] / max_h

            # String to store file
            f_name = "{} {:.3f} {:.3f} {:.3f} {:.3f}".format(0, b_center_x, b_center_y, b_width, b_height)

            print_buffer.append(f_name)
        return print_buffer
        

    def __save_annotations(self, name, print_buffer):
        with open("{}.txt".format(name) , "w") as _file:
            for line in print_buffer:
                _file.write(line+"\n")

            _file.close()
    
    def save_data(self, image, bounding_box_list, name):
        
        image_path = self.path+ r'\image'
        label_path = self.path+ r'\label'
        
        train_image_path = image_path + r'\train'
        val_image_path = image_path + r'\val'
        
        train_label_path = label_path + r'\train'
        val_label_path = label_path + r'\val'
        
        self.__check_path_exists(train_image_path)
        self.__check_path_exists(val_image_path)
        
        self.__check_path_exists(train_label_path)
        self.__check_path_exists(val_label_path)
        
        print_buffer = self.create_annotations(bounding_box_list)
        
        val_or_train = random.randint(0, 11)
        
        # put in the training folder
        if val_or_train <= 7:
            cv.imwrite(train_image_path+ '\\' +name+".jpg", image)
            self.__save_annotations(train_label_path + "\\"+ name, print_buffer)
        else:
            cv.imwrite(val_image_path+ "\\" +name+".jpg", image)
            self.__save_annotations(val_label_path + "\\" + name, print_buffer)
        
        
        